Inspired by Deep Learning for Computer Vision with Python [Rosebrock]  
Chapter 20  
Pretrained ImageNet Classifiers

In [ ]:
from os.path import expanduser

import glob
import tensorflow as tf
import numpy as np
import cv2

In [ ]:
# define a dictionary of models
MODELS = {
    "vgg16": {
        'network': tf.keras.applications.VGG16,
        'input_shape': (224, 224),
        'preprocess': tf.keras.applications.imagenet_utils.preprocess_input,
    },

    "vgg19": {
        'network': tf.keras.applications.VGG19,
        'input_shape': (224, 224),
        'preprocess': tf.keras.applications.imagenet_utils.preprocess_input,
    },

    "resnet": {
        'network': tf.keras.applications.ResNet50,
        'input_shape': (224, 224),
        'preprocess': tf.keras.applications.imagenet_utils.preprocess_input,
    },

    "inception": {
        'network': tf.keras.applications.InceptionV3,
        'input_shape': (299, 299),
        'preprocess': tf.keras.applications.inception_v3.preprocess_input,
    },

    "xception": {
        'network': tf.keras.applications.Xception,
        'input_shape': (299, 299),
        'preprocess': tf.keras.applications.inception_v3.preprocess_input,
    },
}

In [ ]:
# prepare the network
NETWORK_CHOICE = 'inception'

network = MODELS[NETWORK_CHOICE]['network']
input_shape = MODELS[NETWORK_CHOICE]['input_shape']
preprocess = MODELS[NETWORK_CHOICE]['preprocess']
print(NETWORK_CHOICE, input_shape)

# load the network weights from disk
model = network(weights='imagenet')

In [ ]:
# get list of image files
file_list = glob.glob(expanduser('~/dlcv/code/example_images/*.jpg'))
file_list.sort()

for image_file in file_list:
    print(image_file)

    # load the input image
    image = tf.keras.preprocessing.image.load_img(image_file, target_size=input_shape)

    # resize to the required input dimensions
    image = tf.keras.preprocessing.image.img_to_array(image)

    # expand the dimension of the image array from (rows, cols, 3) to (1, rows, cols, 3)
    image = np.expand_dims(image, axis=0)

    # preprocess the image using the appropriate function
    image = preprocess(image)

    # classify the image
    preds = model.predict(image)
    P = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=3)

    # display the predictions and scores
    for (i, (class_name, class_description, score)) in enumerate(P[0]):
        print(f'{i + 1}. {class_description}: {score * 100:.2f}%')

    # load and display the image and the top prediction
    disp_image = cv2.imread(image_file)
    (class_name, class_description, score) = P[0][0]
    cv2.putText(
        disp_image,
        f'{class_description}',
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        (0, 255, 0),
        2,
    )
    cv2.imshow(image_file, disp_image)
    cv2.waitKey(10)

# pause then close all display windows
cv2.waitKey(30000)
cv2.destroyAllWindows()